In [7]:
import pyvista as pv
import numpy as np
from scipy.spatial import KDTree
from collections import defaultdict, deque

def build_vertex_adjacency(mesh: pv.PolyData) -> dict[int, set[int]]:
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    adjacency: dict[int, set[int]] = defaultdict(set)

    for i, j, k in faces:
        adjacency[i].update((j, k))
        adjacency[j].update((i, k))
        adjacency[k].update((i, j))

    return adjacency

def get_sorted_neighbors_by_curvature(vertex_id: int,
                                      adjacency: dict[int, set[int]],
                                      curvature: np.ndarray) -> list[int]:
    """
    Get neighboring vertices of a given vertex, sorted by decreasing curvature.

    Parameters:
        vertex_id (int): The index of the query vertex.
        adjacency (dict[int, set[int]]): Adjacency map from build_vertex_adjacency.
        curvature (np.ndarray): Array of curvature values for all vertices.

    Returns:
        list[int]: Neighboring vertex indices sorted by descending curvature.
    """
    neighbors = list(adjacency[vertex_id])
    neighbors_sorted = sorted(neighbors, key=lambda idx: curvature[idx], reverse=True)
    return neighbors_sorted


def max_unvisited_curvature_point(curvature: np.ndarray, visited_points: set | list) -> int:
    visited_set = set(visited_points)
    
    if len(visited_set) >= len(curvature):
        raise Exception("All points have been visited.")

    # Use masked array to ignore visited points efficiently
    mask = np.ones_like(curvature, dtype=bool)
    mask[list(visited_set)] = False
    unvisited_curvatures = np.ma.masked_array(curvature, ~mask)

    return int(np.argmax(unvisited_curvatures))

# ====== Three functions to check whether a point is on edge ======

'''
The three functions are implemented based on algorithm of section 2 of
    "Edge and Corner Detection for Unorganized 3D Point Clouds with
    Application to Robotic Welding"

Pipeline along each function:
    1. Find k-nearest neighbors using KDTree.
    2. Compute the centroid of the neighbors.
    3. Compare the centroid-to-query distance against λ * min neighbor distance.
'''

def knn_KDTree(points: np.ndarray, query_id: int, k: int) -> np.ndarray:
    tree = KDTree(points)
    query_point = points[query_id]
    distances, indices = tree.query(query_point, k=k+1)
    return indices[1:]  # Exclude the query point itself

def compute_centroid_of_neighbors(points: np.ndarray, neighbor_ids: np.ndarray) -> np.ndarray:
    neighbors = points[neighbor_ids]
    centroid = np.mean(neighbors, axis=0)
    return centroid

def check_edge_point(points: np.ndarray, query_id: int, neighbor_ids: np.ndarray,
                           centroid: np.ndarray, lam: float) -> bool:
    query_point = points[query_id]
    neighbor_points = points[neighbor_ids]
    distances = np.linalg.norm(neighbor_points - query_point, axis=1)
    Z_i = np.min(distances)
    distance_centroid = np.linalg.norm(centroid - query_point)
    return distance_centroid > lam * Z_i

# =================================================================

In [8]:
def laplacian_smoothing(mesh: pv.PolyData, iterations=3):
    
    # Extract face indices (reshape from PyVista's format)
    faces = mesh.faces.reshape((-1, 4))[:, 1:]  # Drop the leading "3" of each triangle

    adjacency = build_vertex_adjacency(mesh)

    points = mesh.points.copy()

    for _ in range(iterations):
        new_points = points.copy()
        for i in range(len(points)):
            neighbors = list(adjacency[i])
            if not neighbors:
                continue
            neighbor_coords = points[neighbors]
            new_points[i] = neighbor_coords.mean(axis=0)
        points = new_points  # update for next iteration

    # Create a new smoothed mesh to avoid altering the original
    smoothed_mesh = pv.PolyData(points, mesh.faces)
    return smoothed_mesh

In [9]:
import numpy as np
from scipy.spatial import KDTree

def is_edge_point_pipeline(points: np.ndarray, query_id: int, k: int, lam: float) -> bool:
    # Execute full pipeline
    neighbor_ids = knn_KDTree(points, query_id, k)
    centroid = compute_centroid_of_neighbors(points, neighbor_ids)
    return check_edge_point(points, query_id, neighbor_ids, centroid, lam)


In [10]:
# Đọc mesh từ file .obj
mesh = pv.read('../CG_dataset/cube_subdivide.obj')
# mesh = laplacian_smoothing(mesh)  
curvature = mesh.curvature(curv_type='mean')
clim = [np.percentile(curvature, 3), np.percentile(curvature, 90)]

adj = build_vertex_adjacency(mesh)

points = mesh.points

k = 20               # Số điểm lân cận
lam = 2.0            # Hệ số λ

num_start_points = 10

visited_points = set()
edge_points = set()

# for i in range(mesh.n_points):
#     if is_edge_point_pipeline(points, i, k, lam):
#         edge_points.add(i)

In [14]:
max_point = max_unvisited_curvature_point(curvature, visited_points)
edge_points.add(max_point)

In [15]:
edge_points

{0}

In [16]:
p = pv.Plotter()
p.add_mesh(mesh, show_edges=True, scalars=curvature, clim=clim)
p.add_mesh(mesh.points[(list(edge_points))], color='red', point_size=10)  

p.show()


Widget(value='<iframe src="http://localhost:35203/index.html?ui=P_0x781c1942da30_1&reconnect=auto" class="pyvi…

In [17]:
# import nbformat

# # 📌 BƯỚC 1: Đặt tên file notebook và file xuất ra
# notebook_file = "edge_detection_unorganized_pc.ipynb"     # 👉 đổi thành tên notebook của bạn
# output_file = "extract_code_from_ipynb.py"      # 👉 tên file .py để lưu code

# # 📌 BƯỚC 2: Đọc file notebook
# with open(notebook_file, "r", encoding="utf-8") as f:
#     nb = nbformat.read(f, as_version=4)

# # 📌 BƯỚC 3: Giả sử ô hiện tại đang chạy là ô cuối cùng (có thể điều chỉnh nếu cần)
# current_cell_index = len(nb.cells) - 1

# # 📌 BƯỚC 4: Lấy code từ tất cả các cell code phía trên
# code_above = ""
# for i in range(current_cell_index):
#     cell = nb.cells[i]
#     if cell.cell_type == "code":
#         code_above += cell.source + "\n\n"

# # 📌 BƯỚC 5: Lưu vào file .py
# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(code_above)

# print(f"✅ Đã lưu mã từ các cell phía trên vào file: {output_file}")
